<a href="https://colab.research.google.com/github/kalp121212/DS203_Project/blob/main/Shanghai_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
